# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [5]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [7]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [8]:
qty_per_customer = data.groupby(['CustomerID', 'ProductName']).sum()
qty_per_customer.drop(['SalesID','ProductID'], axis=1, inplace=True)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [9]:
matrix = pd.pivot_table(qty_per_customer, index='ProductName', values='Quantity', columns='CustomerID')
matrix.fillna(0, inplace=True)
matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
squareform(pdist(matrix, 'euclidean'))

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
distances = pd.DataFrame(1 / 
                         (1 + squareform(pdist(matrix.T, 'euclidean'))), index=matrix.columns, columns=matrix.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [19]:
customer = distances.loc[33,:]
customer.sort_values(ascending=False,inplace=True)
customer[1:6]

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [18]:
lst = []
list(map(lambda x: lst.append(int(x)), list((customer[1:6].index))))
sim_customers = qty_per_customer.loc[lst]
sim_customers

Quantity
CustomerID ProductName                                
200        Bacardi Breezer - Tropical                1
           Bagel - Plain                             1
           Bar - Granola Trail Mix Fruit Nut         1
           Beef - Ground Medium                      1
           Beef - Montreal Smoked Brisket            1
...                                                ...
464        Vaccum Bag 10x13                          2
           Wine - Crozes Hermitage E.                1
           Wine - Sogrape Mateus Rose                1
           Wine - White, Colubia Cresh               1
           Wine - Wyndham Estate Bin 777             1

[325 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [14]:
agg_products = sim_customers.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
agg_products

,Quantity
ProductName,
Mayonnaise - Individual Pkg,4
Lamb - Ground,4
Remy Red,4
Potatoes - Idaho 100 Count,3
Scallops - 10/20,3
...,...
"Hickory Smoke, Liquid",1
Hinge W Undercut,1
Hot Chocolate - Individual,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [15]:
top5 = pd.merge(agg_products, matrix.loc[:,33], on='ProductName')
top5.loc[top5[33] == 0][0:5]

,Quantity,33
ProductName,,
Mayonnaise - Individual Pkg,4,0.0
Lamb - Ground,4,0.0
Remy Red,4,0.0
Scallops - 10/20,3,0.0
Flavouring - Orange,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [35]:
customers = list(set(data['CustomerID']))
def recommendator(customers):
    recom = {}
    for e in customers:
        customer = distances.loc[e,:]
        customer.sort_values(ascending=False, inplace=True)
        lst = []
        list(map(lambda x: lst.append(int(x)), list((customer[1:6].index))))
        sim_customers = qty_per_customer.loc[lst]
        agg_products = sim_customers.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
        productlist = pd.merge(agg_products, matrix.loc[:,e], on='ProductName')
        top5 = productlist.loc[productlist[e] == 0][0:5]
        recom[e] = list(top5.index)
    return recom
print(recommendator(customers))

{83973: ['Pastry - Choclate Baked', 'Flour - Pastry', 'Puree - Passion Fruit', 'Lettuce - Spring Mix', 'Broom - Corn'], 59399: ['Anchovy Paste - 56 G Tube', 'Ezy Change Mophandle', 'Garlic - Peeled', 'Beans - Wax', 'Broom - Corn'], 92168: ['Wiberg Super Cure', 'Tuna - Salad Premix', 'Sage - Ground', 'Apricots Fresh', 'Beef - Inside Round'], 49159: ['Turnip - White, Organic', 'Peas - Frozen', 'Lamb - Ground', 'Beer - Blue', 'Beef - Ground Medium'], 18441: ['V8 - Berry Blend', 'Whmis - Spray Bottle Trigger', 'Mussels - Cultivated', 'Ecolab - Lime - A - Way 4/4 L', 'Milk - 2%'], 22536: ['Hersey Shakes', 'Bread Crumbs - Panko', 'Muffin Batt - Choc Chk', 'Spinach - Baby', 'Wine - White Cab Sauv.on'], 86028: ['Mayonnaise - Individual Pkg', 'Pail With Metal Handle 16l White', 'Sunflower Seed Raw', 'Sausage - Breakfast', 'Appetizer - Mini Egg Roll, Shrimp'], 75791: ['Peas - Frozen', 'Placemat - Scallop, White', 'Beef - Ground Medium', 'Milk - 2%', 'Sausage - Liver'], 96272: ['Bread - Rye', 'Li

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [44]:
pd.DataFrame.from_dict(recom, orient='index')

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
...,...,...,...,...,...
77814,Cheese - Victor Et Berthold,Cake - Cake Sheet Macaroon,Sea Bass - Whole,Squid U5 - Thailand,Guinea Fowl
32753,Grouper - Fresh,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa
20476,Soup - Campbells Tomato Ravioli,"Pasta - Detalini, White, Fresh","Cheese - Boursin, Garlic / Herbs",Beans - Kidney White,Wine - Blue Nun Qualitatswein
67582,Cake - Mini Cheesecake,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [49]:
distances = pd.DataFrame(1 / 
                         (1 + squareform(pdist(matrix.T, 'cosine'))), index=matrix.columns, columns=matrix.columns)
recommendator(customers)
pd.DataFrame.from_dict(recom, orient='index')

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
...,...,...,...,...,...
77814,Cheese - Victor Et Berthold,Cake - Cake Sheet Macaroon,Sea Bass - Whole,Squid U5 - Thailand,Guinea Fowl
32753,Grouper - Fresh,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa
20476,Soup - Campbells Tomato Ravioli,"Pasta - Detalini, White, Fresh","Cheese - Boursin, Garlic / Herbs",Beans - Kidney White,Wine - Blue Nun Qualitatswein
67582,Cake - Mini Cheesecake,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini
